## Import Dependencies

In [3]:
from highcharts_core.chart import Chart
from matplotlib.cbook import boxplot_stats
import pandas as pd

## Prepare the Configuration

## Populate the Dataframe

In [22]:
df = pd.read_csv('../../../../pygada/explorative_data_analysis/demo_dataset_filtered_3D.txt')
df

C:\Users\vandekgu\AppData\Local\Temp\ipykernel_19224\1237096719.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../../../../pygada/explorative_data_analysis/demo_dataset_filtered_3D.txt')


,Unnamed: 0,id,date,x_m_L72,y_m_L72,top_m_mv,basis_m_mv,parameter,detection_condition,value,unit,source,matrix
0,0,31063144,2021-01-06,237521.000,204927.0000,0.5,0.7,PFHxStotal,<,0.2,µg/kg ds,OVAM,Soil
1,1,31063147,2021-01-06,237521.000,204927.0000,0.5,0.7,PFPA,<,0.2,µg/kg ds,OVAM,Soil
2,2,31063151,2021-01-06,237521.000,204927.0000,0.5,0.7,8:2 diPAP,<,0.2,µg/kg ds,OVAM,Soil
3,3,31063157,2021-01-06,237521.000,204927.0000,0.5,0.7,PFPeS,<,0.2,µg/kg ds,OVAM,Soil
4,4,31063159,2021-01-06,237521.000,204927.0000,0.5,0.7,PFNS,<,0.2,µg/kg ds,OVAM,Soil
...,...,...,...,...,...,...,...,...,...,...,...,...,...
418894,234786,PFC3B-B11004-26,2022-01-19,154930.246,213400.2709,10.4,10.8,6:2/8:2 diPAP,<,1.0,µg/kg ds,Lantis,Soil
418895,234787,PFC3B-B11004-26,2022-01-19,154930.246,213400.2709,10.4,10.8,10:2 FTS,<,1.0,µg/kg ds,Lantis,Soil
418896,234789,PFC3B-B11004-26,2022-01-19,154930.246,213400.2709,10.4,10.8,MePFBSA,<,1.0,µg/kg ds,Lantis,Soil
418897,234790,PFC3B-B11004-26,2022-01-19,154930.246,213400.2709,10.4,10.8,MePFBSAA,<,1.0,µg/kg ds,Lantis,Soil


In [36]:
duplicate_values = df[df.duplicated(subset=(['date','x_m_L72', 'y_m_L72', 'top_m_mv', 'basis_m_mv', 'parameter']))]
duplicate_values

,Unnamed: 0,id,date,x_m_L72,y_m_L72,top_m_mv,basis_m_mv,parameter,detection_condition,value,unit,source,matrix,new_id
1751,1751,31415934,2021-08-18,106002.0,201050.0,0.9,1.40,PFBA,<,0.05,µg/kg ds,OVAM,Soil,3416
1768,1768,31415951,2021-08-18,106002.0,201050.0,0.9,1.40,PFOSAtotal,<,0.05,µg/kg ds,OVAM,Soil,3416
1770,1770,31415953,2021-08-18,106002.0,201050.0,0.9,1.40,HFPO-DA,=,0.29,µg/kg ds,OVAM,Soil,3416
1779,1779,31415962,2021-08-18,106002.0,201050.0,0.0,0.25,PFOStotal,<,0.05,µg/kg ds,OVAM,Soil,3415
1796,1796,31415979,2021-08-18,106002.0,201050.0,0.0,0.25,PFPA,<,0.05,µg/kg ds,OVAM,Soil,3415
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
412418,416738,35028456,2023-01-18,205092.0,198350.0,3.0,4.00,PFOA,<,1.00,ng/l,OVAM,Groundwater,12056
412458,416778,35028833,2023-01-18,205153.0,198389.0,0.0,0.00,PFOA,<,1.00,ng/l,OVAM,Groundwater,12057
412469,416789,35028853,2023-01-18,205153.0,198389.0,0.0,0.00,PFOS,<,1.00,ng/l,OVAM,Groundwater,12057
412502,416822,35028928,2023-01-18,205090.0,198379.0,0.0,0.00,PFOS,<,1.00,ng/l,OVAM,Groundwater,12055


In [151]:
data_series = []
def boxplot_matrix(df, matrix, count):
    data = df[df['matrix']==matrix] #data = df[(df['matrix']==matrix) & (df['source']=='VMM')]
    unit = data['unit'].unique()[0]
    data['new_id'] = data.groupby(['date','x_m_L72', 'y_m_L72', 'top_m_mv', 'basis_m_mv']).ngroup()

    data = data.pivot_table(index='new_id', columns='parameter', values='value')
    data = data.loc[:, data.columns.notna()]


    df_boxplot_stats = pd.DataFrame({'whislo':[], 'q1':[], 'med':[], 'q3':[], 'whishi':[]})
    df_boxplot_outliers = pd.DataFrame({'parameter':[], 'value':[]})

    for i in list(data.columns):
        boxplot = boxplot_stats(data[i].dropna())
        boxplot = pd.DataFrame(boxplot)
        boxplot = boxplot.drop(['mean', 'iqr', 'cilo', 'cihi', 'fliers'], axis = 1)
        whishi = boxplot.pop('whishi')
        boxplot.insert(4, 'whishi', whishi)
        df_boxplot_stats = pd.concat([df_boxplot_stats,boxplot])
        outliers = data[i][(data[i] < boxplot['whislo'].values[0]) | (data[i] > boxplot['whishi'].values[0])]
        df_boxplot_outliers = pd.concat([df_boxplot_outliers,pd.DataFrame({'parameter':i, 'value':outliers.values})])

    df_boxplot_stats = df_boxplot_stats.transpose()
    df_boxplot_stats = df_boxplot_stats.set_axis(list(data.columns.values), axis=1)
    df_boxplot_stats = df_boxplot_stats.replace(0, 0.000000001)

    columns = list(df_boxplot_stats.columns)
    data = list(df_boxplot_stats[x].values.tolist() for x in columns)


    index_list = []
    outliers_list = []
    for i in df_boxplot_outliers['parameter'].values.tolist():
        index_list.append(columns.index(i))
    df_boxplot_outliers['index'] = index_list

    for i in range(len(df_boxplot_outliers)):
        outliers_list.append([df_boxplot_outliers['index'].values.tolist()[i], df_boxplot_outliers['value'].values.tolist()[i]])

    data = {
        "name": "Boxplots",
        "data": data,
        "tooltip": {
            "headerFormat": '<em>Parameter {point.key}</em><br/>'
        },
        'yAxis': f'{count}',
        "type": 'boxplot',
    "turboThreshold": 0},{
        "name": "Outliers",
        "data": outliers_list,
        "type": 'scatter',
        "marker": {
            "fillColor": 'white',
            "lineWidth": 0.5,
            "lineColor": "lightblue"
        },
        "tooltip": {
            "pointFormat": 'Concentration: {point.y} ' f'{unit}'
    },
    'yAxis': f'{count}'}

    return data, outliers_list, unit, columns

data_gw, outliers_list_gw, unit_gw, columns_gw = boxplot_matrix(df, 'Groundwater', count = 0)
data_series.extend(data_gw)
data_soil, outliers_list_soil, unit_soil, columns_soil = boxplot_matrix(df, 'Soil', count = 1)
data_series.extend(data_soil)
columns = columns_gw
columns.extend(x for x in columns_soil if x not in columns)

data_series

C:\Users\vandekgu\AppData\Local\Temp\ipykernel_19224\3146297161.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['new_id'] = data.groupby(['date','x_m_L72', 'y_m_L72', 'top_m_mv', 'basis_m_mv']).ngroup()
C:\Users\vandekgu\AppData\Local\Temp\ipykernel_19224\3146297161.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['new_id'] = data.groupby(['date','x_m_L72', 'y_m_L72', 'top_m_mv', 'basis_m_mv']).ngroup()


[{'name': 'Boxplots',
  'data': [[1e-09, 4.0, 4.0, 10.0, 18.0],
   [1e-09, 1.0, 2.0, 10.0, 23.0],
   [2.0, 2.0, 2.0, 2.0, 2.0],
   [1e-09, 1.0, 10.0, 21.0, 51.0],
   [10.0, 10.0, 10.0, 10.0, 10.0],
   [1e-09, 10.0, 10.0, 20.0, 30.0],
   [1e-09, 10.0, 10.0, 20.0, 20.0],
   [1e-09, 1.0, 2.0, 10.0, 23.0],
   [2.0, 2.0, 2.0, 2.0, 2.0],
   [10.0, 10.0, 10.0, 10.0, 10.0],
   [1e-09, 1.0, 1.0, 10.0, 21.0],
   [1.0, 1.0, 1.0, 1.0, 1.0],
   [1e-09, 1.0, 1.0, 10.0, 20.0],
   [1e-09, 1.0, 10.0, 10.0, 23.0],
   [4.0, 4.0, 4.0, 4.0, 4.0],
   [1.0, 1.0, 1.0, 1.0, 1.0],
   [1e-09, 4.0, 10.0, 10.0, 12.0],
   [10.0, 10.0, 10.0, 20.0, 20.0],
   [1e-09, 1.0, 5.0, 10.0, 23.0],
   [10.0, 10.0, 10.0, 10.0, 10.0],
   [1.0, 1.0, 1.0, 1.0, 1.0],
   [1e-09, 10.0, 10.0, 50.0, 110.0],
   [1e-09, 10.0, 10.0, 50.0, 110.0],
   [2.0, 2.0, 2.0, 2.0, 2.0],
   [1e-09, 1.0, 4.0, 10.0, 23.0],
   [1e-09, 2.0, 10.0, 10.0, 20.0],
   [10.0, 10.0, 10.0, 10.0, 10.0],
   [1e-09, 10.0, 32.0, 140.0, 330.0],
   [1e-09, 8.0, 12.0, 6

## Create the Series

In [164]:
options_kwargs = {
    'title': {
        'text': 'Box Plot per Parameter'
    },
    'plotOptions': {
            'series': {
                'turboThreshold': 0,
                'boostThreshold': 0
        }
    },
    'turboThreshold': 0,
    'legend': {
        'enabled': True
    },

    'xAxis': {
        'categories': columns,
        'title': {
            'text': 'Parameter'
        }
    },

    'yAxis': [{
        'type': 'logarithmic',
        'minorTickInterval': 0.1,
        'title': {
            'text': f'Groundwater concentration in log {unit_gw}'
        },
        'prototype': {'allowNegativeLog': True},
        'positiveValuesOnly': False,
        'height': '50%'
    }, {
        'type': 'logarithmic',
        'minorTickInterval': 0.1,
        'title': {
            'text': f'Soil concentration in log {unit_soil}'
        },
        'prototype': {'allowNegativeLog': True},
        'positiveValuesOnly': False,
        'top': '50%',
        'height': '50%',
        'offset': 0,
    }]
}

## Assemble the Chart

In [165]:
chart = Chart(options = options_kwargs)
chart.add_series(*data_series)

## Display the Chart

In [166]:
chart.display(container = 'highcharts_container')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>